In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


In [10]:

train_dataloader = DataLoader(training_data, batch_size=128,num_workers=8,pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=128,num_workers=8,pin_memory=True)


In [11]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
# Hyperparameters

learning_rate = 1e-3
batch_size = 64
epochs = 5

In [13]:
# Optimization Loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300108  [  128/60000]
loss: 2.286610  [12928/60000]
loss: 2.271689  [25728/60000]
loss: 2.270035  [38528/60000]
loss: 2.244520  [51328/60000]
Test Error: 
 Accuracy: 24.6%, Avg loss: 2.238779 

Epoch 2
-------------------------------
loss: 2.239113  [  128/60000]
loss: 2.217417  [12928/60000]
loss: 2.205479  [25728/60000]
loss: 2.206968  [38528/60000]
loss: 2.169852  [51328/60000]
Test Error: 
 Accuracy: 34.9%, Avg loss: 2.163882 

Epoch 3
-------------------------------
loss: 2.166451  [  128/60000]
loss: 2.132273  [12928/60000]
loss: 2.120583  [25728/60000]
loss: 2.120448  [38528/60000]
loss: 2.072544  [51328/60000]
Test Error: 
 Accuracy: 47.4%, Avg loss: 2.061414 

Epoch 4
-------------------------------
loss: 2.062626  [  128/60000]
loss: 2.012709  [12928/60000]
loss: 2.000399  [25728/60000]
loss: 1.992453  [38528/60000]
loss: 1.940139  [51328/60000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.918138 

Epoch 5
------------------------